In [2]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

In [3]:
data = pd.read_csv('Data/telecom_churn.csv')

In [4]:
data.head()

,state,account length,area code,phone number,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,...,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [5]:
    data.drop(['state','voice mail plan', 'phone number'], axis=1,inplace=True)

In [6]:
data['international plan'] = data['international plan'].map({'yes': 1,
                                                             'no': 0})

In [7]:
data.head()


,account length,area code,international plan,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
0,128,415,0,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,107,415,0,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,137,415,0,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,84,408,1,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,75,415,1,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   account length          3333 non-null   int64  
 1   area code               3333 non-null   int64  
 2   international plan      3333 non-null   int64  
 3   number vmail messages   3333 non-null   int64  
 4   total day minutes       3333 non-null   float64
 5   total day calls         3333 non-null   int64  
 6   total day charge        3333 non-null   float64
 7   total eve minutes       3333 non-null   float64
 8   total eve calls         3333 non-null   int64  
 9   total eve charge        3333 non-null   float64
 10  total night minutes     3333 non-null   float64
 11  total night calls       3333 non-null   int64  
 12  total night charge      3333 non-null   float64
 13  total intl minutes      3333 non-null   float64
 14  total intl calls        3333 non-null   

In [9]:
y = data['churn'].astype('int');

In [10]:
X = data.drop('churn',axis=1)

In [11]:
y.shape, X.shape


((3333,), (3333, 17))

In [12]:
from sklearn.model_selection import train_test_split, cross_val_score
import numpy as np

In [13]:
X_train,X_valid, y_train, y_valid =  train_test_split(X,y, test_size=0.3, random_state=17)

In [14]:
X_train.shape, X_valid.shape

((2333, 17), (1000, 17))

In [15]:
first_tree = DecisionTreeClassifier(random_state=17)

In [16]:
cross_val_score(first_tree, X_train, y_train,cv=5)

array([0.9143469 , 0.91220557, 0.92077088, 0.90772532, 0.91416309])

In [17]:
np.mean(cross_val_score(first_tree, X_train, y_train,cv=5))

0.9138423504976518

In [18]:
from sklearn.neighbors import KNeighborsClassifier

In [19]:
first_knn = KNeighborsClassifier()

In [20]:
np.mean(cross_val_score(first_knn, X_train, y_train,cv=5))

0.8671274043984523

In [68]:
## Настраиваем максимальную гулбину

In [21]:
from sklearn.model_selection import GridSearchCV

In [24]:
tree_params = {'max_depth': np.arange(1, 11), 'max_features':[.5, .7, 1]}


In [25]:
print(tree_params)


{'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]), 'max_features': [0.5, 0.7, 1]}


In [27]:
tree_grid = GridSearchCV(first_tree,tree_params,cv=5,n_jobs=-1)

In [30]:
%%time
tree_grid.fit(X_train,y_train);

Wall time: 323 ms


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=17), n_jobs=-1,
             param_grid={'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
                         'max_features': [0.5, 0.7, 1]})

In [31]:
tree_grid.best_params_

{'max_depth': 6, 'max_features': 0.7}

In [32]:
tree_grid.best_score_

0.9391366681677404

In [34]:
## Настраиваем параметры для KNN



In [35]:
knn_params = {'n_neighbors':  range(5, 30, 5) }#+ list(range(50, 100, 10))}

In [37]:
knn_grid = GridSearchCV(first_knn,knn_params,cv=5,n_jobs=-1)

In [39]:
%%time
knn_grid.fit(X_train, y_train);

Wall time: 809 ms


GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': range(5, 30, 5)})

In [40]:
knn_grid.best_estimator_,knn_grid.best_params_, knn_grid.best_score_


(KNeighborsClassifier(n_neighbors=10), {'n_neighbors': 10}, 0.8701289391697531)

In [41]:
## проверка


In [44]:
tree_valid_pred= tree_grid.predict(X_valid)

In [45]:
tree_grid.score(X_valid,y_valid) 

0.936

In [46]:
from sklearn.metrics import accuracy_score

In [47]:
accuracy_score(y_valid,tree_valid_pred)

0.936

In [49]:
from sklearn.tree import export_graphviz

In [51]:
export_graphviz(tree_grid.best_estimator_, out_file='telecom_tree2.dot',
               feature_names=X.columns, filled=True,)

In [52]:
!dot -Tpng telecom_tree2.dot -o telecom_tree2.png

"dot" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.
